<a href="https://colab.research.google.com/github/jayesh-bansal/NLP/blob/main/Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - 08

* Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]

(i) Build the Transformer Model on above dataset

(ii) Train the model using 20, 60, 70 epochs

 (iii) After training, use the model to generate new text by feeding it an initial seed text

(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
def build_model(vocab_size, max_len):
    embed_dim = 64
    num_heads = 2
    ff_dim = 32
    inputs = tf.keras.Input(shape=(max_len - 1,))
    embedding_layer = Embedding(vocab_size, embed_dim, input_length=max_len - 1)(inputs)


    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(embedding_layer, training=True)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(20, activation="relu")(x)
    x = Dense(vocab_size, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [ ]:

model = build_model(total_words, max_sequence_len)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs_list = [20, 60, 70]
for epochs in epochs_list:
    print(f"Training with {epochs} epochs...")
    model.fit(X, y, epochs=epochs, verbose=2)


Training with 20 epochs...
Epoch 1/20
2/2 - 4s - 2s/step - accuracy: 0.0577 - loss: 3.8884
Epoch 2/20
2/2 - 0s - 93ms/step - accuracy: 0.0577 - loss: 3.7060
Epoch 3/20
2/2 - 0s - 23ms/step - accuracy: 0.0385 - loss: 3.6694
Epoch 4/20
2/2 - 0s - 28ms/step - accuracy: 0.0577 - loss: 3.6360
Epoch 5/20
2/2 - 0s - 22ms/step - accuracy: 0.0385 - loss: 3.6540
Epoch 6/20
2/2 - 0s - 30ms/step - accuracy: 0.0577 - loss: 3.6275
Epoch 7/20
2/2 - 0s - 29ms/step - accuracy: 0.0577 - loss: 3.5746
Epoch 8/20
2/2 - 0s - 23ms/step - accuracy: 0.0769 - loss: 3.5675
Epoch 9/20
2/2 - 0s - 23ms/step - accuracy: 0.0769 - loss: 3.5697
Epoch 10/20
2/2 - 0s - 23ms/step - accuracy: 0.0769 - loss: 3.5276
Epoch 11/20
2/2 - 0s - 24ms/step - accuracy: 0.0577 - loss: 3.5382
Epoch 12/20
2/2 - 0s - 29ms/step - accuracy: 0.0962 - loss: 3.5083
Epoch 13/20
2/2 - 0s - 64ms/step - accuracy: 0.0962 - loss: 3.4765
Epoch 14/20
2/2 - 0s - 23ms/step - accuracy: 0.0962 - loss: 3.4570
Epoch 15/20
2/2 - 0s - 23ms/step - accuracy: 0

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


seed_text = "Once upon a time"
print(generate_text(seed_text, 10, max_sequence_len))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Once upon a time there was a little girl named red riding hood hood
